# Multi-step CNN Models - Multiple Parallel Input and Multi-step Output 

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


#### split a multivariate sequence into samples

In [2]:
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

## 1. Data preparation

#### define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

#### convert to [rows, columns] structure

In [4]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

#### horizontally stack column

In [5]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))

#### choose a number of time steps

In [6]:
n_steps_in, n_steps_out = 3, 2

#### convert into input/output

In [7]:
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

#### flatten output

In [8]:
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

#### the dataset knows the number of features, e.g. 2

In [9]:
n_features = X.shape[2]

## 2. CNN Model

#### define model

In [10]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

#### fit model

In [11]:
model.fit(X, y, epochs=7000, verbose=0)

#### demonstrate prediction

In [12]:
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 90.35666  95.54947 185.58359 100.62263 105.61126 206.20253]]
